In [1]:
import sys
import os
to_add = os.getcwd().split('/scripts')[0]
if not to_add in sys.path:
    sys.path.insert(0, to_add)

import diff_classifier.aws as aws
import pandas as pd
import boto3
import matplotlib.pyplot as plt
import numpy as np
import statsmodels.api as sm
from scipy.optimize import leastsq
from scipy.stats import sem
import ferretfit.ferretfit as ff

# Example use of ferret_fit

In [ ]:
prefix = 'N_P1_R1'
folder = 'ferret_tracking'
test = ff.ferret_fit(folder, prefix)

# Example code to loop over multiple tracks

In [ ]:
#This cell creates a list of csv names over which we will loop

s3 = boto3.resource('s3')
my_bucket = s3.Bucket('ccurtis.data')

all_files = []
for object_summary in my_bucket.objects.filter(Prefix=folder):
    all_files.append(object_summary.key)
    
all_csvs = []
for num in all_files:
    if 'csv' in num:
        all_csvs.append(num.split('/')[1].split('.')[0])

In [ ]:
all_csvs[0:-3]

In [ ]:
#And this cell does the brunt of the work.

d = {'deviation': [], 'range': [], 'rsd': [], 'amplitude': [], 'period': []}
df = pd.DataFrame(data=d)

for prefix in all_csvs[0:-3]:

    ystd, rang, rsd, amp, per, pawcount, pawdens, cross, crossdens, stride, stridestd = ff.ferret_fit(folder, prefix)
    df = df.append({'deviation':ystd, 'range':rang, 'rsd':rsd, 'amplitude':amp,
                    'pawcount':pawcount, 'pawdensity':pawdens, 'period':per,
                    'cross':cross, 'crossdensity':crossdens, 'stride':stride, 'stridestd':stridestd, 'run':prefix}, ignore_index=True)

In [ ]:
df

In [ ]:
#This cell allows the user to perform averages of a parameter of interest after analysis and compare groups.

measure = 'period'

print('Mean wonky {}: {} +/- {}'.format(measure, np.average(df[0:15][measure]), sem(df[0:15][measure])))
print('Mean normal {}: {} +/- {}'.format(measure, np.average(df[15:30][measure]), sem(df[15:30][measure])))

np.average(df[0:15]['period'])

In [ ]:
#This cell saves the analyzed data to a csv file.
df.to_csv('ferret_stats.csv')

stats = 'ferret_stats.csv'
aws.upload_s3(stats, '{}/{}'.format(folder, stats), bucket_name='ccurtis.data')

In [ ]:
np.arcsin()